In [25]:
import sys

import re

import glob

import os

import os.path

import cv2

import random

import numpy as np

import matplotlib

import matplotlib.pyplot as plt

import pandas as pd


In [26]:
def read_image(file_path):

    image   = cv2.imread(file_path, cv2.IMREAD_COLOR) 

   
    image = cv2.resize(image, (ROWS, COLS), interpolation = cv2.INTER_CUBIC)

    return image


In [27]:
def make_params(file_path):
    
    basename = os.path.basename(file_path)
    file_number = int(re.sub(r'\D','',basename))
    laser_strength =file_number //30
    
    if 'Cu_' in basename :
        df = pd.read_csv('soturon.csv')
        df_Cu = df[df["material"] == "Cu"]
        Cu_array = df_Cu.to_numpy()
        matparam = np.delete(Cu_array, 0, 1)
    else :
        print(file_path)

    params = np.empty(0)
    
    
    params = np.append(matparam, laser_strength)
    
    return params

    

In [28]:
def preprocess(images):

    count = len(images)

    data = np.ndarray((count, ROWS, COLS, CHANNELS), dtype = np.uint8)



    for i, image_file in enumerate(images):

        image = read_image(image_file)
    
        image_rgb = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        
        params = make_params(image_file)
        
        numar = np.empty([19, COLS, ROWS])
        
        for j in range(0, 19):
            
            numar[j] = np.full((COLS, ROWS), params[j])
            
       
        image_rgb_b = np.concatenate([image_rgb.transpose(2, 0, 1), numar],axis=0)
        
        #image_rgb = np.append(image_rgb.T, np.array([numar]), axis=0)
        
        data[i] = image_rgb_b.T
        
    

    data = data.astype('float32')

    data = data/255.0



    return data
            
        

In [29]:
#Deep Learning module

import tensorflow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping
from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical


Model Initializer¶

In [30]:
def Material_model():



    model = Sequential()

    model.add(Conv2D(2, kernel_size=(5, 5), activation='relu', kernel_initializer='he_normal', input_shape=(ROWS, COLS, CHANNELS)))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', kernel_initializer='he_normal'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))

    model.add(Dropout(0.2))


    model.add(Dense(2, kernel_initializer='he_normal'))

    model.add(Dense(2, activation='softmax'))

    model.summary()

    model.compile(loss=objective, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [31]:
ROWS = 28

COLS = 28

CHANNELS = 22


In [32]:
train_inputdirectory ='CuDataset/TrainData'
test_inputdirectory='CuDataset/TestData'


In [33]:
Da_train = glob.glob(train_inputdirectory + "/Damage/*.jpg")

Un_train = glob.glob(train_inputdirectory + "/NotDamage/*.jpg")


train_images = Da_train + Un_train

random.shuffle(train_images)

In [34]:
len(train_images)

4301

In [35]:
train_labels = []

for i in train_images:

    if 'Da_' in i :

        train_labels.append(0)

    if 'NotDa_' in i :

        train_labels.append(1)


In [36]:
Da_test = glob.glob(test_inputdirectory + "/Damage/*.jpg")

NotDa_test = glob.glob(test_inputdirectory + "/NotDamage/*.jpg")



test_images = Da_test + NotDa_test

random.shuffle(test_images)



#正解ラベルのリスト作成

test_labels = []

for i in test_images:

    if 'Da_' in i :

        test_labels.append(0)

    if 'NotDa_' in i :

        test_labels.append(1)

In [37]:
train_data = preprocess(train_images)
test_data=preprocess(test_images)

CuDataset/TrainData/Damage\Damage._22_8484142.jpg


UnboundLocalError: local variable 'matparam' referenced before assignment